In [1]:
import ast
from typing import Annotated, TypedDict
from uuid import uuid4

from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.messages import AIMessage, HumanMessage, ToolCall
from langchain_core.tools import tool
from langchain_ollama import ChatOllama

from langgraph.graph import START, StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

/home/anseh/.pyenv/versions/langchain/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
@tool
def calculator(query: str) -> str:
    """계산기, 수식만 입력받습니다."""
    return ast.literal_eval(query)


search = DuckDuckGoSearchRun()
tools = [search, calculator]
model = ChatOllama(model="granite3.3:2b", temperature=0.1).bind_tools(tools)

In [3]:
class State(TypedDict):
    messages: Annotated[list, add_messages]


def model_node(state: State) -> State:
    res = model.invoke(state["messages"])
    return {"messages": res}


def first_model(state: State) -> State:
    query = state["messages"][-1].content
    search_tool_call = ToolCall(
        name="duckduckgo_search", args={"query": query}, id=uuid4().hex
    )
    return {"messages": AIMessage(content="", tool_calls=[search_tool_call])}


builder = StateGraph(State)
builder.add_node("first_model", first_model)
builder.add_node("model", model_node)
builder.add_node("tools", ToolNode(tools))
builder.add_edge(START, "first_model")
builder.add_edge("first_model", "tools")
builder.add_conditional_edges("model", tools_condition)
builder.add_edge("tools", "model")

graph = builder.compile()

In [4]:
input = {
    "messages": [HumanMessage("미국의 제30대 대통령이 사망했을 때 몇 살이었나요?")]
}

for c in graph.stream(input):
    print(c)

{'first_model': {'messages': AIMessage(content='', additional_kwargs={}, response_metadata={}, id='4edfd900-98ac-416f-b970-831cbbbfa659', tool_calls=[{'name': 'duckduckgo_search', 'args': {'query': '미국의 제30대 대통령이 사망했을 때 몇 살이었나요?'}, 'id': 'fa0b75878fd442699f7dcbf3d61e8edb', 'type': 'tool_call'}])}}
{'tools': {'messages': [ToolMessage(content='미국의 역사를 통틀어 정치는 대부분 정당이 지배했다. 1789년 헌법이 발효된 당시에는 제대로 조직된 정당이 존재하지 않았으며, 이로 인해 현재까지 헌법은 정당에 대해 언급하지 않는다. 1921년 3월 4일 하딩은 대통령으로서 취임되었고, 1923년 8월 2일 사망할 때까지 대통령직을 지냈으며 하딩의 사망에 쿨리지는 대통령이 되었다. 쿨리지는 하딩이 사망할 때 자신 집에 있었다. 8월 3일 새벽 2시 47분 공증인 그의 부친 및 가족들과 응접실에서 대통령직을 선서했고, 쿨리지는 워싱턴 D. C.로 돌아와 윌리엄 하워드 태프트 대법관에 의하여 다시 선서했다. 1924년 미국 대통령 선거에서 선출되었다. 제25차 개정안에는 "대통령이 직위에서 해임되거나 사망하거나 사임할 경우 부통령이 대통령이 된다"고 명시되어 있습니다. 도널드 트럼프 미국 대통령이 한국 정부의 핵추진잠수함 건조를 승인했으나 현실화되기는 어렵다는 지적이 나오고 있다. 한반도 정세와 투입되는 비용 등을 고려할 때 핵추진잠수함이 전략적으로 좋은 선택이 아니라는 지적도 제기된다. 링컨은 암살당한 최초의 미국 대통령이다. 그는 1865년 4월 14일 워싱턴 DC에서 열린 연극 공연에 참석하던 중 존 윌크스 부스에게 총을 맞았다. 링컨은 4월 15일 다음 날 아침 사망했다. 그의 흑인 권리 지지가 살해